#### DNN기반 이진분류 모델 구현
- 데이터셋   : iris.csv
- 피쳐/속성  : 4개 Sepal_Length, Sepal-Width, Petal_Length,Petal.Width
- 타겟/라벨  : 1개 Setosa 외 나머지
- 학습-방법  : 지도학습 > 분류 > 2중분류
- 알고리즘   : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score,BinaryF1Score,BinaryConfusionMatrix
from torchinfo import summary
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [2]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수화 
def verCHeck():
    print(f'Pytorch.v {torch.__version__}')
    print(f'Pandas.v {pd.__version__}')
verCHeck()

Pytorch.v 2.4.1
Pandas.v 2.0.3


In [3]:
### 데이터 로딩
DATA_FILE='../data/iris.csv'
irisDF=pd.read_csv(DATA_FILE)
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [4]:
### 타겟변경 => 정수화, 클레스 3개 => 2개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [5]:
irisDF['variety']=(irisDF['variety']=='Setosa')
irisDF['variety']=irisDF['variety'].astype('int')
irisDF

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0
146,6.3,2.5,5.0,1.9,0
147,6.5,3.0,5.2,2.0,0
148,6.2,3.4,5.4,2.3,0


[2] 모델 클레스 설계 및 정의
- 클레스목적 : iris 데이터를 학습 및 추론 목적
- 클레스이름 : IrisBCFModel
- 부모클레스 : nn.Module
- 매개__변수 : 층별 입출력 개수 고정하기 떄문에 필요 없음!
- 속성__필드 : 
- 기능__역할 : _ _init_ _() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)
- 클레스 구조
    * 입력층 : 입력 4개(피쳐)  출력 10개(퍼셉트론/뉴런 10개 존제)
    * 은닉층 : 입력 10개       출력 5개(퍼셉트론/뉴런 5개 존제)
    * 출력층 : 입력 5개       출력 1개(퍼셉트론/뉴런 1개 존제 : 2진분류)

- 활성화함수
    * 클레스 형태 ==> nn.MESLoss,nn.ReLU ==>_ _init_ _()함수
    * 함수 형태 ==> torch.nn.functional 아래

In [6]:
class IrisBCFModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hd_layer=nn.Linear(10,5)
        self.out_layer=nn.Linear(5,1)

    #순방향 학습 진행 매서드
    def forward(self,input_data):
        # 입력층 
        y=F.relu(self.in_layer(input_data))
        # 은닉층 : 10개의 숫자 값(>=0)
        y=F.relu(self.hd_layer(y))
        # 출력층 : 5개의 숫자 값 => 이진분류
        return F.sigmoid(self.out_layer(y))

In [7]:
## 모델 인스턴스 생성
model=IrisBCFModel()
print(model)

IrisBCEModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [8]:
#모델 사용 메모리 정보 확인
summary(model,input_size=(17,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisBCEModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3]클레스 설계 및 정의

In [9]:
class IrisDataset(Dataset):
    
    def __init__(self,featureDF,targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 반환
        return featureTS,targetTS

In [10]:
## 데이터셋 인스턴스 생성 
# DF에서 피쳐와 타겟 추출
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[irisDF.columns[-1:]]
# 커스텀 데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF,targetDF)
# 데이터 로더 인스턴스 생성
irisDL=DataLoader(irisDS)
for feature,label in irisDL:
    print(feature.shape,label.shape,feature,label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습준비
- 학습횟수 : EPOCH 
- 배치크기 : BATCH_SIZE 
- 위치지정 : DEVICE 
- 학습률 : LR 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001~0.1

In [11]:
### 학습 진행 관련 설정
EPOCH=100
BATCH_SIZE=10
BACH_CNT=irisDF.shape[0]/BATCH_SIZE
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화, (손실함수 ,성능지표)

In [12]:
#모델 인스턴스 
model=IrisBCFModel()

In [13]:
#DS과 DL 인스턴스
X_train,X_test,y_train,y_test=train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=1)
print(f'{X_train.shape},{X_test.shape},{X_val.shape}')
print(f'{y_train.shape},{y_test.shape},{y_val.shape}')
trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)
# 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS,batch_size=BATCH_SIZE)

(84, 4),(38, 4),(28, 4)
(84, 1),(38, 1),(28, 1)


- 최적화, 손실함수 인스턴스

In [14]:
# 최적화 인스턴스 => W,b텐서 즉, model.paramiters()전달
optimizer=optim.Adam(model.parameters(),lr=LR)
# 손실함수 인스턴스 => 분류 => 이진분류 BinaryCrossEntropyLoss => BCELoss 
#                            예측값은 확률값으로 전달 => sigmoid() AF 처리 후 전달
reqLoss=nn.BCELoss()

[5] 학습진행

In [15]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY,SCORE_HISTORY=[[],[]],[[],[]]
CNT=len(trainDL)
for epoch in range(EPOCH):
    # 학습모드로 모델 설정
    model.train()
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total,score_total=0,0
    for featureTS,targetTS in trainDL:
        # 학습진행
        pre_y=model(featureTS)
        # 손실계산
        loss=reqLoss(pre_y,targetTS)
        loss_total+=loss
        #성능평가 계산
        score=BinaryF1Score()(pre_y,targetTS)
        score_total+=score
        # 최적화진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # IF 에포크당 검증 한다면
    # 모델 검증모드
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val=model(val_featureTS)
        # 손실
        loss_val=reqLoss(pre_val,val_targetTS)
        # 성능평가
        score_val=BinaryF1Score()(pre_val,val_targetTS)
    # 에포크당 손실값과 성능 평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)
    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')

[0/100]
- [TRAIN] LOSS : 0.5452248454093933 SCORE : 0.0
[1/100]
- [TRAIN] LOSS : 0.5375556945800781 SCORE : 0.0
[2/100]
- [TRAIN] LOSS : 0.5303506851196289 SCORE : 0.0
[3/100]
- [TRAIN] LOSS : 0.5237919688224792 SCORE : 0.0
[4/100]
- [TRAIN] LOSS : 0.5179089307785034 SCORE : 0.0
[5/100]
- [TRAIN] LOSS : 0.5125063061714172 SCORE : 0.0
[6/100]
- [TRAIN] LOSS : 0.50751793384552 SCORE : 0.0
[7/100]
- [TRAIN] LOSS : 0.5028852224349976 SCORE : 0.0
[8/100]
- [TRAIN] LOSS : 0.4985595643520355 SCORE : 0.0
[9/100]
- [TRAIN] LOSS : 0.4944726526737213 SCORE : 0.0
[10/100]
- [TRAIN] LOSS : 0.4905572831630707 SCORE : 0.0
[11/100]
- [TRAIN] LOSS : 0.48675018548965454 SCORE : 0.0
[12/100]
- [TRAIN] LOSS : 0.4830038547515869 SCORE : 0.0
[13/100]
- [TRAIN] LOSS : 0.4792662262916565 SCORE : 0.0
[14/100]
- [TRAIN] LOSS : 0.47549968957901 SCORE : 0.0
[15/100]
- [TRAIN] LOSS : 0.4716739058494568 SCORE : 0.0
[16/100]
- [TRAIN] LOSS : 0.46778541803359985 SCORE : 0.0
[17/100]
- [TRAIN] LOSS : 0.463811159133911

- 학습결과 체크 => 학습과 검증의 Loss 변화 ,성능 변화